In [54]:
import cv2
import numpy as np




cap = cv2.VideoCapture(0)               # '0' is the webcam's ID. usually it is 0 or 1. 'cap' is the video object.
cap.set(15, 4)                         # '15' references video's brightness. '-4' sets the brightness.


while True:

    ret, im = cap.read()
    if im is None:
        break
    #cv2.imshow('orig step 0',im)
    #convert to hsv then split into 3 values
    hsv = cv2.cvtColor(im,cv2.COLOR_BGR2HSV)
    #FINDS THE GREEN VALUES
    mask = cv2.inRange(hsv, (40,25,25),(100,255,255))
    #filters out non green values
    imask = mask>0
    green = np.zeros_like(im, np.uint8)
    green[imask] = im[imask]
    #thresholds the frame and convert all of the green to white and everything else to black
    hsv = cv2.cvtColor(green, cv2.COLOR_BGR2HSV)
    h,s,v = cv2.split(hsv)
    th, threshed = cv2.threshold(s,50,255,cv2.THRESH_BINARY)
    #finds the contours on the threshold image
    cnts = cv2.findContours(threshed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2]
    canvas = im.copy()
    cnts = sorted(cnts,key = cv2.contourArea)
    #if a green section was detected then this segments executes
    if cnts:
        #this finds the largest contour which should be the placeholder image.
        #this can be modified to find the second/third/fourth largest image.
        cnt = cnts[-1]
        arclen = cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, 0.02* arclen, True)
        #cv2.drawContours(canvas, [cnt], -1, (255,0,0), 1, cv2.LINE_AA)
        #cv2.drawContours(canvas, [approx], -1, (0, 0, 255), 1, cv2.LINE_AA)
        #cv2.imshow('square detection final',canvas)
        
        #creates a blank black image
        blank = np.zeros_like(im, np.uint8)
        #draws the approximate contour onto the blank image then converts it to grayscale. This converts the red contour line to white
        cv2.drawContours(blank, [approx], -1, (0, 0, 255), 1, cv2.LINE_AA)
        blank = cv2.cvtColor(blank,cv2.COLOR_BGR2GRAY)
        blank = np.float32(blank)
        
        #detects the corners and stores them into a list of corners
        dst = cv2.cornerHarris(blank,2,3,0.04)
        ret, dst = cv2.threshold(dst,0.1*dst.max(),255,0)
        dst = np.uint8(dst)
        ret, labels, stats, centroids = cv2.connectedComponentsWithStats(dst)
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.001)
        corners = cv2.cornerSubPix(blank,np.float32(centroids),(5,5),(-1,-1),criteria)
        #this creates a list of corners again
        tempCorners = []
        for i in range(1, len(corners)):
            tempCorners.append(corners[i])
        #if the detected green segment is rectangular/square then we proceed to this segment
        if len(tempCorners) == 4:
            #print('Square:')
            #overlay image
            input = cv2.imread('cola.jpg')
            #cv2.imshow('input',input)
            #input image size, this can be determined based off the actual image size dynamically.
            input_pts = np.float32([[1000,0],[0,0],[0,562],[1000,562]])
            #uses the list of corners again but in a format that works with warp perspective
            output_pts = np.float32([tempCorners[1],tempCorners[0],tempCorners[2],tempCorners[3]])
            # sort to organize the output points
            
            
            
            #warps the original image to that of the placeholder item
            M = cv2.getPerspectiveTransform(input_pts,output_pts)
            out = cv2.warpPerspective(input,M,(canvas.shape[1], canvas.shape[0]),flags=cv2.INTER_LINEAR)
            
            #overlays the input image.
            canvas = cv2.addWeighted(canvas,0.7,out,0.3,0)
        canvas[dst>0.1*dst.max()]=[0,0,255]
        
        #original image.
        cv2.imshow('original', im)
        #shows the final image with contours and the overlaid image. The contours can easily be removed.
        cv2.imshow('Final', canvas)
        

    k = cv2.waitKey(30) & 0xff                              # press [Esc] to exit.
    if k == 27:
        break

cv2.destroyAllWindows()



In [16]:
input_pts = np.sort(input_pts, 1)
            if input_pts[0][0] > input_pts[1][0]:
                input_pts[0][0],  input_pts[1][0] =  input_pts[1][0], input_pts[0][0]
                
            if input_pts[2][0] > input_pts[3][0]:
                input_pts[2][0],  input_pts[3][0] =  input_pts[3][0], input_pts[2][0]

IndentationError: unexpected indent (<ipython-input-16-c81e29b37e99>, line 2)

In [ ]:
output_pts = np.sort(output_pts, 1)
            if output_pts[0][0] > output_pts[1][0]:
                output_pts[0][0],  output_pts[1][0] =  output_pts[1][0], output_pts[0][0]
                
            if output_pts[2][0] > output_pts[3][0]:
                output_pts[2][0],  output_pts[3][0] =  output_pts[3][0], output_pts[2][0]
            
            